# Explore ATT&CK Detection Telemetry

This notebook explores the modern detection model in ATT&CK by walking the hierarchy:

`Technique → Detection Strategy → Analytic → Log Source References → Data Components`


## Goals

- Load ATT&CK content locally (download latest STIX 2.1 bundles).
- Enrich techniques with detection strategies and analytics.
- Explore analytic log source references and the data-component IDs they point to.
- Optionally inline full data-component objects under each log source.


## Import ATT&CK Client


In [1]:
from attackcti import MitreAttackClient

## Import Libraries


In [ ]:
import json

import altair as alt
import pandas as pd

alt.renderers.enable('default')


RendererRegistry.enable('default')

## Initialize Client (Download Latest STIX 2.1)


In [3]:
lift = MitreAttackClient.from_attack_stix_data()
print('mode:', lift.mode, 'spec_version:', lift.spec_version)


mode: local spec_version: 2.1


## Get Techniques (Detection Enrichment)

`enrich_detections=True` attaches the following custom properties:

- `x_attackcti_detection_strategies`
- `x_attackcti_analytics`
- `x_attackcti_log_sources` (each entry includes `x_mitre_data_component_ref`)


In [4]:
techniques = lift.get_techniques(enrich_detections=True)
len(techniques)


898

## Build a Table of Log Source References per Technique


In [5]:
records = []

for t in techniques:
    d = json.loads(t.serialize()) if hasattr(t, 'serialize') else t

    domain = (d.get('x_mitre_domains') or [None])[0]
    technique_id = d.get('id')
    technique_name = d.get('name')
    platforms = d.get('x_mitre_platforms') or []

    # Kill chain phases are a list of dicts: {'phase_name': ..., 'kill_chain_name': ...}
    tactics = [p.get('phase_name') for p in (d.get('kill_chain_phases') or []) if isinstance(p, dict) and p.get('phase_name')]

    strategies = d.get('x_attackcti_detection_strategies') or []

    log_source_names: set[str] = set()
    data_component_refs: set[str] = set()

    for strategy in strategies:
        for analytic in (strategy.get('x_attackcti_analytics') or []):
            for log_source in (analytic.get('x_attackcti_log_sources') or []):
                name = log_source.get('name')
                if isinstance(name, str) and name:
                    log_source_names.add(name)
                ref = log_source.get('x_mitre_data_component_ref')
                if isinstance(ref, str) and ref:
                    data_component_refs.add(ref)

    records.append(
        {
            'domain': domain,
            'technique': technique_name,
            'technique_id': technique_id,
            'platforms': platforms,
            'tactics': tactics,
            'detection_strategies_count': len(strategies),
            'log_sources': sorted(log_source_names) or None,
            'log_sources_count': len(log_source_names),
            'data_component_refs_count': len(data_component_refs),
        }
    )

df = pd.DataFrame.from_records(records)
df.head()


,domain,technique,technique_id,platforms,tactics,detection_strategies_count,log_sources,log_sources_count,data_component_refs_count
0,enterprise-attack,Extra Window Memory Injection,attack-pattern--0042a9f5-f053-4769-b3ef-9ad018...,[Windows],"[defense-evasion, privilege-escalation]",1,"[WinEventLog:Security, WinEventLog:Sysmon, etw...",3,3
1,enterprise-attack,Scheduled Task,attack-pattern--005a06c6-14bf-4118-afa0-ebcd8a...,[Windows],"[execution, persistence, privilege-escalation]",1,"[WinEventLog:Security, WinEventLog:Sysmon]",2,5
2,enterprise-attack,Socket Filters,attack-pattern--005cc321-08ce-4d17-b1ea-cb5275...,"[Linux, macOS, Windows]","[defense-evasion, persistence, command-and-con...",1,"[NSM:Flow, OpenBSM:AuditTrail, WinEventLog:Sys...",7,6
3,enterprise-attack,Archive via Utility,attack-pattern--00f90846-cbd1-4fc5-9233-df5c2b...,"[Linux, macOS, Windows]",[collection],1,"[WinEventLog:Security, WinEventLog:Sysmon, aud...",5,4
4,enterprise-attack,VNC,attack-pattern--01327cde-66c4-4123-bf34-5f258d...,"[Linux, Windows, macOS]",[lateral-movement],1,"[NSM:Flow, NSM:firewall, WinEventLog:Security,...",8,4


## Visualize Coverage Across Domains

These charts summarize how detection telemetry coverage (via log source references) is distributed across domains.


In [6]:
domain_counts = df.groupby('domain', dropna=False).size().reset_index(name='techniques')
domain_counts


,domain,techniques
0,enterprise-attack,691
1,ics-attack,83
2,mobile-attack,124


In [7]:
alt.Chart(domain_counts).mark_bar().encode(
    x=alt.X('techniques:Q', title='Technique count'),
    y=alt.Y('domain:N', sort='-x', title='Domain'),
    color='domain:N'
).properties(height=140)


alt.Chart(...)

## Visualize Techniques With and Without Log Sources

This mirrors the legacy “with/without data sources” view, but using `log_sources_count` instead.


In [8]:
dist2 = pd.DataFrame(
    {
        'Techniques': ['Without log sources', 'With log sources'],
        'Count': [int((df['log_sources_count'] == 0).sum()), int((df['log_sources_count'] > 0).sum())],
    }
)

bars = alt.Chart(dist2).mark_bar().encode(x='Techniques', y='Count', color='Techniques').properties(width=260, height=280)
text = bars.mark_text(align='center', baseline='bottom', dy=-2).encode(text='Count')
(bars + text)


alt.LayerChart(...)

## Techniques by Platform (With Log Sources)

This shows which platforms have the most techniques with at least one log source reference.


In [9]:
df_platform = df[df['log_sources_count'] > 0].copy()
df_platform = df_platform.explode('platforms')
df_platform_counts = df_platform.groupby('platforms', dropna=False).size().reset_index(name='techniques')
df_platform_counts = df_platform_counts.sort_values('techniques', ascending=False)
df_platform_counts.head(15)


,platforms,techniques
11,Windows,472
13,macOS,355
5,Linux,354
2,ESXi,118
3,IaaS,104
0,Android,102
6,Network Devices,100
8,Office Suite,78
12,iOS,76
7,None,70


In [10]:
alt.Chart(df_platform_counts).mark_bar().encode(
    x=alt.X('techniques:Q', title='Technique count'),
    y=alt.Y('platforms:N', sort='-x', title='Platform')
).properties(height=260)


alt.Chart(...)

## Top Log Source Names

Log source references include a `name` field (for example `WinEventLog:Sysmon`). This chart shows the most common log source names across techniques.


In [11]:
df_ls = df[df['log_sources'].notna()].copy()
df_ls = df_ls.explode('log_sources')
ls_counts = df_ls.groupby('log_sources').size().reset_index(name='techniques')
ls_counts = ls_counts.sort_values('techniques', ascending=False).head(20)
ls_counts


,log_sources,techniques
103,WinEventLog:Sysmon,420
233,macos:unifiedlog,347
122,auditd:SYSCALL,332
102,WinEventLog:Security,278
58,NSM:Flow,129
231,macos:osquery,94
2,AWS:CloudTrail,89
209,linux:syslog,81
7,Application Vetting,78
64,Network Traffic,70


In [12]:
alt.Chart(ls_counts).mark_bar().encode(
    x=alt.X('techniques:Q', title='Technique count'),
    y=alt.Y('log_sources:N', sort='-x', title='Log source')
).properties(height=420)


alt.Chart(...)

## Techniques With and Without Log Source References


In [13]:
dist = pd.DataFrame(
    {
        'Techniques': ['Without log sources', 'With log sources'],
        'Count': [int((df['log_sources_count'] == 0).sum()), int((df['log_sources_count'] > 0).sum())],
    }
)

bars = alt.Chart(dist).mark_bar().encode(x='Techniques', y='Count', color='Techniques').properties(width=260, height=280)
text = bars.mark_text(align='center', baseline='bottom', dy=-2).encode(text='Count')
(bars + text)


alt.LayerChart(...)

## Example: Print the Enriched Hierarchy for One Technique


In [14]:
technique = json.loads(techniques[0].serialize()) if hasattr(techniques[0], 'serialize') else techniques[0]

details = technique.get('x_attackcti_detection_strategies', [])

for strategy in details:
    print(f"Detection strategy: {strategy.get('name')} ({strategy.get('id')})")
    for analytic in strategy.get('x_attackcti_analytics', []):
        print(f"  Analytic: {analytic.get('name')} ({analytic.get('id')})")
        for log_source in analytic.get('x_attackcti_log_sources', []):
            comp_ref = log_source.get('x_mitre_data_component_ref')
            print(f"    Log source: {log_source.get('name')} - {log_source.get('channel')} (component ref {comp_ref})")


Detection strategy: Detection Strategy for Extra Window Memory (EWM) Injection on Windows (x-mitre-detection-strategy--1a8d87f1-48ca-4929-a5cc-2b2a03983f12)
  Analytic: Analytic 0608 (x-mitre-analytic--6ec034ac-289d-48d1-b310-021dfbf7087b)
    Log source: WinEventLog:Sysmon - EventCode=10 (component ref x-mitre-data-component--1887a270-576a-4049-84de-ef746b2572d6)
    Log source: etw:Microsoft-Windows-Win32k - SetWindowLong, SetClassLong, NtUserMessageCall, SendNotifyMessage, PostMessage (component ref x-mitre-data-component--9bde2f9d-a695-4344-bfac-f2dce13d121e)
    Log source: WinEventLog:Security - EventCode=4688 (component ref x-mitre-data-component--3d20385b-24ef-40e1-9f56-f39750379077)


## Optional: Inline Full Data Component Objects

If you also want the full `x-mitre-data-component` objects attached under each log source, call:

`get_techniques(enrich_detections=True, enrich_data_components=True)`

This adds `x_attackcti_data_component` under each log source (when resolvable).


In [15]:
techniques_with_components = lift.get_techniques(enrich_data_components=True)

technique = json.loads(techniques_with_components[0].serialize()) if hasattr(techniques_with_components[0], 'serialize') else techniques_with_components[0]

details = technique.get('x_attackcti_detection_strategies', [])

for strategy in details:
    print(f"Detection strategy: {strategy.get('name')} ({strategy.get('id')})")
    for analytic in strategy.get('x_attackcti_analytics', []):
        print(f"  Analytic: {analytic.get('name')} ({analytic.get('id')})")
        for log_source in analytic.get('x_attackcti_log_sources', []):
            comp_ref = log_source.get('x_mitre_data_component_ref')
            print(f"    Log source: {log_source.get('name')} - {log_source.get('channel')} (component ref {comp_ref})")

            comp = log_source.get('x_attackcti_data_component')
            if isinstance(comp, dict):
                print(f"      Data component: {comp.get('name')} ({comp.get('id')})")


Detection strategy: Detection Strategy for Extra Window Memory (EWM) Injection on Windows (x-mitre-detection-strategy--1a8d87f1-48ca-4929-a5cc-2b2a03983f12)
  Analytic: Analytic 0608 (x-mitre-analytic--6ec034ac-289d-48d1-b310-021dfbf7087b)
    Log source: WinEventLog:Sysmon - EventCode=10 (component ref x-mitre-data-component--1887a270-576a-4049-84de-ef746b2572d6)
      Data component: Process Access (x-mitre-data-component--1887a270-576a-4049-84de-ef746b2572d6)
    Log source: etw:Microsoft-Windows-Win32k - SetWindowLong, SetClassLong, NtUserMessageCall, SendNotifyMessage, PostMessage (component ref x-mitre-data-component--9bde2f9d-a695-4344-bfac-f2dce13d121e)
      Data component: OS API Execution (x-mitre-data-component--9bde2f9d-a695-4344-bfac-f2dce13d121e)
    Log source: WinEventLog:Security - EventCode=4688 (component ref x-mitre-data-component--3d20385b-24ef-40e1-9f56-f39750379077)
      Data component: Process Creation (x-mitre-data-component--3d20385b-24ef-40e1-9f56-f39750379